### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
constructed_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
print(constructed_url)

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_query = f"&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(constructed_url + page_query, params={"api-key": nyt_api_key})
    
    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        reviews = response.json()
        
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        
        # Print the page that was just retrieved
        print(f"Retrieved page {page + 1}")
        
    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"No results found for page {page + 1}: {e}")
        break



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)


In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Extract the title from the "headline.main" column
def extract_title(headline):
    start_index = headline.find('\u2018') + len('\u2018')
    end_index = headline.find(' Review') + len(' Review')
    return headline[start_index:end_index]

# Apply the extract_title function to create the new "title" column
df_reviews['title'] = df_reviews['headline.main'].apply(extract_title)

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)


In [ ]:
# Create a list from the "title" column using to_list()
title_list = df_reviews['title'].to_list()
# These titles will be used in the query for The Movie Database
# Print each title from title_list on a separate line
for title in title_list:
    print(title)

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)  # Sleep for 10 seconds after every 50 requests

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)

    # Include a try clause to search for the full movie details. 
    # Use the except clause to print out a statement if a movie is not found.
    try:
        # Get movie id
        movie_id = response.json()["results"][0]["id"]

        # Make a request for the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_details_response = requests.get(movie_details_url)

        # Execute "GET" request with url
        movie_details = movie_details_response.json()

        # Extract the genre names into a list
        genres = [genre["name"] for genre in movie_details["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_details["title"],
            "overview": movie_details["overview"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })

        # Print out the title that was found
        print(f"Found details for movie: {title}")

    except IndexError:
        print(f"Movie not found: {title}")

# Print the tmdb_movies_list
for movie in tmdb_movies_list:
    print(movie)



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame table
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

# Print the DataFrame
print(df_tmdb_movies)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb_movies, on="title", how="inner")

# Print the merged DataFrame
print(merged_df)

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for char in chars_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)


In [ ]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)


In [ ]:
# Delete duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Print the DataFrame after deleting duplicates and resetting index
print(merged_df)


In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)